# Parte I: Producción de un conjunto de datos

Autor: Christian Torres

Este notebook está dedicado a obtener todos los datos necesarios para entrenar los modelos de aprendizaje automático de estos ejercicios. Para ello, necesitamos datos climáticos de reanálisis ERA5 y datos observados en la Cordillera Blanca. Los datos de ERA5 pueden descargarse desde [Climate Data Store](https://cds.climate.copernicus.eu/cdsapp#!/home) de Copernicus.
Primero, importamos las dependencias que vamos a necesitar.

```{note}
Antes de ejecutar este notebook, asegúrese de que el kernel de Jupyter (esquina superior izquierda del notebook) se haya configurado para funcionar con el entorno que tienen instalada todas las librerías y dependencias.
```


In [1]:
import xarray as xr
import glob
import pandas as pd

## 1.1. Obtener datos de temperatura del aire

Para entrenar nuestros modelos de aprendizaje automático, necesitamos datos de destino, en este caso, la temperatura del aire a 2 m. Para ello, obtenemos la temperatura superficial observada en las estaciones meteorológicas disponibles. Como un ejercicio práctico usaremos apenas una estación meteorológica disponible. Puedes incluir datos de muchas más estaciones meteorológicas o incluso puedes usar datos obtenidos por sensoriamento remoto. 

In [2]:
OBS_EMA15 = pd.read_csv('../data/EMA15_Tingua.dat', delimiter='\t', index_col=['Fecha'], 
                         parse_dates=['Fecha'])

In [3]:
OBS_EMA15

,Temp,RH2,Precip,QSWin
Fecha,,,,
2014-01-01 01:00:00,12.4,NaN,0.0,NaN
2014-01-01 02:00:00,11.6,NaN,0.0,NaN
2014-01-01 03:00:00,11.1,NaN,0.0,NaN
2014-01-01 04:00:00,10.5,NaN,0.0,NaN
2014-01-01 05:00:00,10.5,NaN,0.0,NaN
...,...,...,...,...
2018-12-12 20:00:00,NaN,NaN,NaN,NaN
2018-12-12 21:00:00,NaN,NaN,NaN,NaN
2018-12-12 22:00:00,NaN,NaN,NaN,NaN


In [4]:
OBS_EMA15_nan = OBS_EMA15[['Temp']].resample('6h').mean().dropna()

In [5]:
OBS_EMA15_nan

,Temp
Fecha,
2014-01-01 00:00:00,11.220000
2014-01-01 06:00:00,15.266667
2014-01-01 12:00:00,23.083333
2014-01-01 18:00:00,15.516667
2014-01-02 00:00:00,13.033333
...,...
2018-12-11 12:00:00,24.346667
2018-12-11 18:00:00,16.898333
2018-12-12 00:00:00,11.650000


Guardamos el conjuntos de temperatura para entrenamiento

In [6]:
OBS_EMA15_nan.to_csv('../data/temp_aws_entrena.csv')

## 1.2. Obtener datos de ERA5 (predictores)

### Adicionar los archivos .nc de superficie

En esta parte adicionamos el camino de todo los archivos .nc usando **`glob`**.

In [7]:
files = sorted(glob.glob('../data/*_CRS.nc'))
print(len(files))
print(files)


5
['../data/ERA5_2014_CRS.nc', '../data/ERA5_2015_CRS.nc', '../data/ERA5_2016_CRS.nc', '../data/ERA5_2017_CRS.nc', '../data/ERA5_2018_CRS.nc']


Ahora vamos a leer todas los archivos **.nc**

In [8]:
ds_sup = xr.open_mfdataset(files)
ds_sup

<xarray.Dataset>
Dimensions:    (longitude: 11, latitude: 17, time: 7304)
Coordinates:
  * longitude  (longitude) float32 -79.0 -78.75 -78.5 ... -77.0 -76.75 -76.5
  * latitude   (latitude) float32 -7.0 -7.25 -7.5 -7.75 ... -10.5 -10.75 -11.0
  * time       (time) datetime64[ns] 2014-01-01 ... 2018-12-31T18:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(1460, 17, 11), meta=np.ndarray>
    z          (time, latitude, longitude) float32 dask.array<chunksize=(1460, 17, 11), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(1460, 17, 11), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-05 21:31:19 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

### Hacemos lo mismo para archivos .nc por niveles

In [9]:
files = sorted(glob.glob('../data/*_CRS_pl.nc'))
print(len(files))
print(files)

5
['../data/ERA5_2014_CRS_pl.nc', '../data/ERA5_2015_CRS_pl.nc', '../data/ERA5_2016_CRS_pl.nc', '../data/ERA5_2017_CRS_pl.nc', '../data/ERA5_2018_CRS_pl.nc']


In [10]:
ds_pl = xr.open_mfdataset(files)
ds_pl

<xarray.Dataset>
Dimensions:    (longitude: 11, latitude: 17, level: 3, time: 7304)
Coordinates:
  * longitude  (longitude) float32 -79.0 -78.75 -78.5 ... -77.0 -76.75 -76.5
  * latitude   (latitude) float32 -7.0 -7.25 -7.5 -7.75 ... -10.5 -10.75 -11.0
  * level      (level) int32 200 500 1000
  * time       (time) datetime64[ns] 2014-01-01 ... 2018-12-31T18:00:00
Data variables:
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(1460, 3, 17, 11), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 dask.array<chunksize=(1460, 3, 17, 11), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-08-02 13:48:01 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

### Datos geográficos de las estaciones meteorológicas

In [11]:
EMA15 = [-77.688333, -9.222450, 2118]

### Seleccionamos para las estaciones meteorológicas disponibles 

In [12]:
ERA5_EMA15_sup = ds_sup.sel(longitude=EMA15[0], latitude= EMA15[1], method='nearest')
ERA5_EMA15_pre = ds_pl.sel(longitude=EMA15[0], latitude= EMA15[1], method='nearest')

### Convetir para dataframe

In [13]:
ERA5_EMA15_sup_df = ERA5_EMA15_sup.to_dataframe().drop(['latitude', 'longitude'], axis=1)
ERA5_EMA15_sup_df


,t2m,z,tp
time,,,
2014-01-01 00:00:00,280.645813,37063.71875,1.158565e-05
2014-01-01 06:00:00,277.901794,37063.71875,0.000000e+00
2014-01-01 12:00:00,279.369354,37063.71875,0.000000e+00
2014-01-01 18:00:00,285.019684,37063.71875,7.392447e-04
2014-01-02 00:00:00,279.970215,37063.71875,1.873057e-04
...,...,...,...
2018-12-30 18:00:00,287.092529,37063.71875,3.051260e-04
2018-12-31 00:00:00,281.842804,37063.71875,4.348811e-05
2018-12-31 06:00:00,278.876556,37063.71875,4.726462e-07


Selecionamos los diferentes niveles de presión

In [14]:
ERA5_EMA15_pre_1000 = ERA5_EMA15_pre.sel(level=1000).to_dataframe().drop(['latitude', 'longitude', 'level'], axis=1)
ERA5_EMA15_pre_1000.columns = ['g_1000', 't_1000']
ERA5_EMA15_pre_500 = ERA5_EMA15_pre.sel(level=500).to_dataframe().drop(['latitude', 'longitude', 'level'], axis=1)
ERA5_EMA15_pre_500.columns = ['g_500', 't_500']
ERA5_EMA15_pre_200 = ERA5_EMA15_pre.sel(level=200).to_dataframe().drop(['latitude', 'longitude', 'level'], axis=1)
ERA5_EMA15_pre_200.columns = ['g_200', 't_200']


Juntamos los dataframes

In [15]:
df_ERA5_all = pd.concat([ERA5_EMA15_sup_df, ERA5_EMA15_pre_1000, ERA5_EMA15_pre_500, ERA5_EMA15_pre_200], axis=1)

In [16]:
df_ERA5_all

,t2m,z,tp,g_1000,t_1000,g_500,t_500,g_200,t_200
time,,,,,,,,,
2014-01-01 00:00:00,280.645813,37063.71875,1.158565e-05,0.008835,297.136017,0.003417,268.610718,0.000093,220.895813
2014-01-01 06:00:00,277.901794,37063.71875,0.000000e+00,0.007724,296.930359,0.002321,267.871521,0.000031,221.523849
2014-01-01 12:00:00,279.369354,37063.71875,0.000000e+00,0.007367,297.062378,0.002287,268.741333,0.000030,221.227905
2014-01-01 18:00:00,285.019684,37063.71875,7.392447e-04,0.008523,297.136017,0.003990,268.285583,0.000041,221.737823
2014-01-02 00:00:00,279.970215,37063.71875,1.873057e-04,0.008609,297.242981,0.003992,269.072021,0.000092,222.624283
...,...,...,...,...,...,...,...,...,...
2018-12-30 18:00:00,287.092529,37063.71875,3.051260e-04,0.009952,296.729767,0.003633,267.876465,0.000052,220.129486
2018-12-31 00:00:00,281.842804,37063.71875,4.348811e-05,0.008959,297.087402,0.002240,268.994110,0.000069,220.172791
2018-12-31 06:00:00,278.876556,37063.71875,4.726462e-07,0.008518,296.977051,0.001803,269.012268,0.000060,219.337357


### Convertir UCT para LT

In [17]:
df_ERA5_AWS_LT = df_ERA5_all.tz_localize('utc').tz_convert('America/Lima')
df_ERA5_AWS_LT

,t2m,z,tp,g_1000,t_1000,g_500,t_500,g_200,t_200
time,,,,,,,,,
2013-12-31 19:00:00-05:00,280.645813,37063.71875,1.158565e-05,0.008835,297.136017,0.003417,268.610718,0.000093,220.895813
2014-01-01 01:00:00-05:00,277.901794,37063.71875,0.000000e+00,0.007724,296.930359,0.002321,267.871521,0.000031,221.523849
2014-01-01 07:00:00-05:00,279.369354,37063.71875,0.000000e+00,0.007367,297.062378,0.002287,268.741333,0.000030,221.227905
2014-01-01 13:00:00-05:00,285.019684,37063.71875,7.392447e-04,0.008523,297.136017,0.003990,268.285583,0.000041,221.737823
2014-01-01 19:00:00-05:00,279.970215,37063.71875,1.873057e-04,0.008609,297.242981,0.003992,269.072021,0.000092,222.624283
...,...,...,...,...,...,...,...,...,...
2018-12-30 13:00:00-05:00,287.092529,37063.71875,3.051260e-04,0.009952,296.729767,0.003633,267.876465,0.000052,220.129486
2018-12-30 19:00:00-05:00,281.842804,37063.71875,4.348811e-05,0.008959,297.087402,0.002240,268.994110,0.000069,220.172791
2018-12-31 01:00:00-05:00,278.876556,37063.71875,4.726462e-07,0.008518,296.977051,0.001803,269.012268,0.000060,219.337357


In [18]:
import datetime
dt1_list = []
for i in range(len(df_ERA5_AWS_LT)):
    dt1_list.append(datetime.datetime(df_ERA5_AWS_LT.index[i].year,
                  df_ERA5_AWS_LT.index[i].month,
                  df_ERA5_AWS_LT.index[i].day,                    
                  df_ERA5_AWS_LT.index[i].hour,
                  df_ERA5_AWS_LT.index[i].minute,
                  df_ERA5_AWS_LT.index[i].second))


df_ERA5_AWS_LT.index = dt1_list
df_ERA5_AWS_LT.index.name = 'Datetime'

In [19]:
df_ERA5_AWS_LT

,t2m,z,tp,g_1000,t_1000,g_500,t_500,g_200,t_200
Datetime,,,,,,,,,
2013-12-31 19:00:00,280.645813,37063.71875,1.158565e-05,0.008835,297.136017,0.003417,268.610718,0.000093,220.895813
2014-01-01 01:00:00,277.901794,37063.71875,0.000000e+00,0.007724,296.930359,0.002321,267.871521,0.000031,221.523849
2014-01-01 07:00:00,279.369354,37063.71875,0.000000e+00,0.007367,297.062378,0.002287,268.741333,0.000030,221.227905
2014-01-01 13:00:00,285.019684,37063.71875,7.392447e-04,0.008523,297.136017,0.003990,268.285583,0.000041,221.737823
2014-01-01 19:00:00,279.970215,37063.71875,1.873057e-04,0.008609,297.242981,0.003992,269.072021,0.000092,222.624283
...,...,...,...,...,...,...,...,...,...
2018-12-30 13:00:00,287.092529,37063.71875,3.051260e-04,0.009952,296.729767,0.003633,267.876465,0.000052,220.129486
2018-12-30 19:00:00,281.842804,37063.71875,4.348811e-05,0.008959,297.087402,0.002240,268.994110,0.000069,220.172791
2018-12-31 01:00:00,278.876556,37063.71875,4.726462e-07,0.008518,296.977051,0.001803,269.012268,0.000060,219.337357


Guardamos el conjuntos de datos ERA5

In [20]:
df_ERA5_AWS_LT.to_csv('../data/ERA5_all.csv')